In [ ]:
# Install the requirements in Google Colab
# !pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face
# from huggingface_hub import login

# login("")

/home/ai-makina/.pyenv/versions/smol-course-2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch


device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-135M-Homer-Chat-Model"
finetune_tags = ["smoltalk", "homer-chat"]


# Generate with the base model

Here we will try out the base model which does not have a chat template. 

In [4]:
# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

user
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku


## Dataset Preparation

We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.

**TRL will format input messages based on the model's chat templates.** They need to be represented as a list of dictionaries with the keys: `role` and `content`,.

## Dataset

https://huggingface.co/datasets/salhernandez/homer-simpson-smoltalk-everyday-conversations

# Split Dataset

In [4]:

# Load the dataset first - using 'everyday-conversations' config for chat training
ds = load_dataset("salhernandez/homer-simpson-smoltalk-everyday-conversations", split="train")
# Split the dataset for training and evaluation
ds = ds.train_test_split(test_size=0.5, seed=42)

print(f"Training samples: {len(ds['train'])}")
print(f"Test samples: {len(ds['test'])}")
print(f"Example conversation: {ds['train'][0]['messages']}")

Training samples: 1130
Test samples: 1130
Example conversation: [{'content': 'Hi there', 'role': 'user'}, {'content': '"D\'oh! Hello! How can I help you today? Woo-hoo! Don\'t mind me, just trying to make sense of all these wires and circuits. Mmm... I\'m not sure if I\'m doing it right. Oh well, at least I didn\'t burn down the lab again. Woo-hoo! And speaking of which, have you tried that new donut place downtown? It\'s got the best chocolate glazed ever!"', 'role': 'assistant'}, {'content': "I'm having trouble with my work schedule. I have a lot of night shifts this week.", 'role': 'user'}, {'content': '"D\'oh! Night shifts? They\'re like trying to find a needle in a haystack while wearing a blindfold and eating a whole box of donuts. And then someone throws a wrench at your head. Woo-hoo!"', 'role': 'assistant'}, {'content': 'Yes, I am. Do you have any tips for staying awake during my shift?', 'role': 'user'}, {'content': "D'oh! Fresh air? What's that? Woo-hoo! I'm not exactly sure

## Configuring the SFTTrainer

The `SFTTrainer` is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.

In [5]:

# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=8,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    eval_steps=50,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training on Apple Silicon
    hub_model_id=finetune_name,  # Set a unique name for your model
    push_to_hub=True,  # Automatically push the model to Hugging Face Hub
    hub_private_repo=False,  # Make the model public
    report_to=None,  # Disable wandb/tensorboard logging for simplicity
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    processing_class=tokenizer,  # Use processing_class instead of tokenizer
)

Truncating eval dataset: 100%|██████████| 1130/1130 [00:00<00:00, 188167.52 examples/s]


## Training the Model

With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.

In [6]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

Step,Training Loss
10,2.274900
20,1.747500
30,1.599000
40,1.532600
50,1.405400
60,1.383300
70,1.389600
80,1.364000
90,1.333100
100,1.295000


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            





Processing Files (1 / 2)                :   0%|          | 18.1kB /  538MB,   ???B/s  





Processing Files (1 / 2)                :   0%|          |  568kB /  538MB, 1.37MB/s  


Processing Files (1 / 2)                :   0%|          | 2.22MB /  538MB, 3.67MB/s  


Processing Files (1 / 2)                :   1%|          | 5.52MB /  538MB, 6.87MB/s  


Processing Files (1 / 2)                :   2%|▏         | 8.27MB /  538MB, 8.25MB/s  








Processing Files (1 / 2)                :   2%|▏         | 10.5MB /  538MB, 6.53MB/s  


Processing Files (1 / 2)                :   3%|▎         | 14.3MB /  538MB, 7.95MB/s  








Processing Files (1 / 2)                :   4%|▎         | 19.8MB /  538MB, 8.25MB/s  


Processing Files (1 / 2)                :   4%|▍         | 20.9MB /  538MB, 8.04MB/s  








Processing Files (1 / 2)                :   5%|▍         | 26.4MB /  538MB, 8.25MB/s  


P

In [ ]:
trainer.push_to_hub(tags=finetune_tags)

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            





Processing Files (1 / 2)                :  28%|██▊       |  151MB /  538MB,   ???B/s  


Processing Files (1 / 2)                :  58%|█████▊    |  310MB /  538MB,  796MB/s  


Processing Files (1 / 2)                :  87%|████████▋ |  470MB /  538MB,  796MB/s  


Processing Files (2 / 2)                : 100%|██████████|  538MB /  538MB,  644MB/s  











Processing Files (2 / 2)                : 100%|██████████|  538MB /  538MB,  323MB/s  
New Data Upload                         : |          |  0.00B /  0.00B,  0.00B/s  
  ...h Talk/sft_output/training_args.bin: 100%|██████████| 6.10kB / 6.10kB            
  ...h Talk/sft_output/model.safetensors: 100%|██████████|  538MB /  538MB            


CommitInfo(commit_url='https://huggingface.co/salhernandez/SmolLM2-135M-Homer-Chat-Model/commit/c6c326d2d8234f2433705058bc77655eb6536954', commit_message='End of training', commit_description='', oid='c6c326d2d8234f2433705058bc77655eb6536954', pr_url=None, repo_url=RepoUrl('https://huggingface.co/salhernandez/SmolLM2-135M-Homer-Chat-Model', endpoint='https://huggingface.co', repo_type='model', repo_id='salhernandez/SmolLM2-135M-Homer-Chat-Model'), pr_revision=None, pr_num=None)

: 

# Generate with fine-tuned model

In [36]:

model_name = "salhernandez/SmolLM2-135M-Homer-Chat-Model"
# model_name = "salhernandez/SmolLM3-3B-homer-chat-sft-peft"
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_name).to(device)

prompt = "Hi"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=1000)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

user
Hi
assistant
"D'oh! Hello! How can I help you today? Woo-hoo! Don't mind me, just trying to make sense of all these buttons on my phone."
